# Imports

In [1]:
import numpy as np
import pandas as pd
import itertools
import math
import random
import sys

import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, chi2, f_classif, GenericUnivariateSelect

In [2]:
random.seed(42)

# Data

In [3]:
train_orig = pd.read_csv('../data/train.csv')
test_orig = pd.read_csv('../data/test.csv')
event_type = pd.read_csv('../data/event_type.csv')
resource_type = pd.read_csv('../data/resource_type.csv')
severity_type = pd.read_csv('../data/severity_type.csv')
log_feature = pd.read_csv('../data/log_feature.csv')

# Functions

In [4]:
def multi_log_loss(predictions, answers):
    m = len(predictions)
    zeros = np.zeros(shape=(m, 3))
    y = pd.DataFrame(zeros, columns=[0, 1, 2])
    for i, c in enumerate(answers):
        y.set_value(i, c, 1)
    return -(1/m) * np.sum(np.sum(y * np.log(predictions)))

In [5]:
def train_test(df, base_split=0.20, recombine=0.6):
    n_splits = int(math.floor(1/base_split))
    nmb_unknown = len(df['location']) * 0.10
    locations = df['location'].value_counts()
    location_labels = sorted(locations.index.tolist())
    unique_location_sets = []
    for i in range(n_splits):
        ctr = 0
        group_labels = []
        while ctr < nmb_unknown:
            group_index = random.randint(0, len(location_labels)-1)
            unknown = location_labels[group_index]
            location_labels[group_index], location_labels[-1] = \
                location_labels[-1], location_labels[group_index]
            location_labels.pop()
            group_labels.append(unknown)
            ctr += locations[unknown]
        unique_location_sets.append(group_labels)
    for unknowns in unique_location_sets:
        msk = df['location'].isin(unknowns)
        unknown_test_locations = df[msk]
        rest = df[~msk]
        train = rest.sample(frac=0.44, random_state=42)
        rest_test = rest[~rest.index.isin(train.index)]
        test = pd.concat([rest_test, unknown_test_locations], axis=0)
        yield train, test

In [6]:
def filter_on_total(x, distribution, total=1, value='Rare'):
    try:
        if distribution[x] < total:
            return value
    except (ValueError, KeyError):
        return np.nan
    else:
        return x

In [7]:
def resource_type_features(df):
    resource_type_dummies = pd.get_dummies(resource_type['resource_type'])
    resource_dummy = pd.concat([resource_type, resource_type_dummies],
                               axis=1)
    resource_grpd = resource_dummy.groupby(resource_dummy.id).sum()
    df_resource = pd.merge(df, resource_grpd, left_on='id', right_index=True)
    return df_resource.drop(['resource_type 1', 'resource_type 2',
                             'resource_type 4',
                             'resource_type 6', 'resource_type 8',
                             'resource_type 10'], axis=1)

In [8]:
def resource_type_event_count(df):

    p = pd.merge(resource_type, event_type, on='id')
    t = p.loc[p['resource_type'] == 'resource_type 2', ['id', 'event_type']] \
        .groupby(by=['id'], as_index=False).count()
    df_two = pd.merge(df, t, on='id', how='left')

    f = p.loc[p['resource_type'] == 'resource_type 4', ['id', 'event_type']] \
        .groupby(by='id', as_index=False).count()
    df_four = pd.merge(df_two, f, on='id', how='left')

    e = p.loc[p['resource_type'] == 'resource_type 8', ['id', 'event_type']] \
        .groupby(by='id', as_index=False).count()
    df_eight = pd.merge(df_four, e, on='id', how='left')

    t = p.loc[p['resource_type'] == 'resource_type 10', ['id', 'event_type']] \
        .groupby(by='id', as_index=False).count()
    df_ten = pd.merge(df_eight, t, on='id', how='left')

    return df_ten

In [9]:
def resource_type_log(df):

    p = pd.merge(resource_type, log_feature, on='id')
    t = p.loc[(p['resource_type'] == 'resource_type 1'), ['id', 'log_feature']] \
        .groupby(by='id', as_index=False).count()
    df_one = pd.merge(df, t, on='id', how='left')

    tw = p.loc[(p['resource_type'] == 'resource_type 2'), ['id', 'log_feature']] \
        .groupby(by='id', as_index=False).count()
    df_two = pd.merge(df_one, tw, on='id', how='left')

    se = p.loc[(p['resource_type'] == 'resource_type 7'), ['id', 'log_feature']] \
        .groupby(by='id', as_index=False).count()
    df_se = pd.merge(df_two, se, on='id', how='left')

    ei = p.loc[(p['resource_type'] == 'resource_type 8'), ['id', 'log_feature']] \
        .groupby(by='id', as_index=False).count()
    df_ei = pd.merge(df_se, ei, on='id', how='left')

    n = p.loc[(p['resource_type'] == 'resource_type 9'), ['id', 'log_feature']] \
        .groupby(by='id', as_index=False).count()
    df_n = pd.merge(df_ei, n, on='id', how='left')

    return df_n

In [10]:
def resource_type_log_sum(df):

    p = pd.merge(resource_type, log_feature, on='id')
    t = p.loc[(p['resource_type'] == 'resource_type 3'), ['id', 'volume']] \
        .groupby(by='id', as_index=False).sum()
    df_one = pd.merge(df, t, on='id', how='left')

    f = p.loc[(p['resource_type'] == 'resource_type 4'), ['id', 'volume']] \
        .groupby(by='id', as_index=False).sum()
    df_f = pd.merge(df, f, on='id', how='left')

    return df_f

In [11]:
def resource_type_log_total(df):

    p = pd.merge(resource_type, log_feature, on='id')

    # resource type 2
    t = p.loc[(p['resource_type'] == 'resource_type 2'), ['id', 'log_feature']] \
        .groupby(by='id', as_index=False).count()

    t.loc[:, 'volume'] = p.loc[(p['resource_type'] == 'resource_type 2'), ['id', 'volume']] \
        .groupby(by='id', as_index=False).sum()['volume']

    t.loc[:, 'total log volume two'] = t['volume'] / t['log_feature']

    df_one = pd.merge(df, t[['id', 'total log volume two']], on='id', how='left')

    # resource type ten
    te = p.loc[(p['resource_type'] == 'resource_type 10'), ['id', 'log_feature']] \
        .groupby(by='id', as_index=False).count()

    te.loc[:, 'volume'] = p.loc[(p['resource_type'] == 'resource_type 10'), ['id', 'volume']] \
        .groupby(by='id', as_index=False).sum()['volume']

    te.loc[:, 'total log volume ten'] = te['log_feature'] * te['volume']

    df_te = pd.merge(df_one, te[['id', 'total log volume ten']], on='id', how='left')

    return df_te

In [12]:
def resource_type_severity(df):

    p = pd.merge(resource_type, severity_type, on='id')
    p.loc[:, 'resource type severity'] = \
        ((p['resource_type'] == 'resource_type 6') &
        (p['severity_type'] == 'severity_type 1')).astype(float)

    t = p[['id', 'resource type severity']].groupby(by='id', as_index=False).median()

    return pd.merge(df, t, on='id', how='left')

In [13]:
def severity_type_features(df):

    severity_type_dummies = pd.get_dummies(severity_type['severity_type'])
    severity_dummy = pd.concat([severity_type, severity_type_dummies],
                               axis=1)

    severity_grpd = severity_dummy \
        .groupby(severity_dummy.id).sum()

    df_severity = pd.merge(df, severity_grpd, left_on='id', right_index=True)
    return df_severity

In [14]:
def severity_high(df):

    p = severity_type[['id', 'severity_type']]

    p.loc[:, 'high log severity'] = \
        (p['severity_type'].isin(['severity_type 3',
                                   'severity_type 4',
                                   'severity_type 5'])).astype(float)
    ret = pd.merge(df, p[['id', 'high log severity']], on='id', how='left')
    return ret

In [15]:
def severity_event(df):

    p = pd.merge(severity_type, event_type, on='id')
    t = p.loc[(p['severity_type'] == 'severity_type 1'), ['id', 'event_type']] \
        .groupby(by='id', as_index=False).count()
    df_one = pd.merge(df, t, on='id', how='left')

    return df_one

In [16]:
def severity_log(df):

    p = pd.merge(severity_type, log_feature, on='id')

    tw = p.loc[(p['severity_type'] == 'severity_type 2'), ['id', 'log_feature']] \
        .groupby(by='id', as_index=False).count()
    df_tw = pd.merge(df, tw, on='id', how='left')

    return df_tw

In [17]:
def severity_log_sum(df):

    p = pd.merge(severity_type, log_feature, on='id')

    t = p.loc[(p['severity_type'] == 'severity_type 2'), ['id', 'log_feature']] \
        .groupby(by='id', as_index=False).count()
    t.loc[:, 'severity_volume'] = p.loc[(p['severity_type'] == 'severity_type 2'), ['id', 'volume']] \
        .groupby(by='id', as_index=False).sum()

    t.loc[:, 'severity_log_volume'] = t['severity_volume'] / t['log_feature']
    df_one = pd.merge(df, t[['id', 'severity_log_volume']], on='id', how='left')

    return df_one

In [18]:
def log_features(df):

    log_table = pd.pivot_table(
        log_feature, values='volume',
        index='id', columns='log_feature',
        aggfunc=np.sum, fill_value=0
    )
    df_log = pd.merge(df, log_table, left_on='id', right_index=True)

    return df_log

In [19]:
def log_feature_volume(df):

    g = log_feature[['id', 'volume']].groupby(by='id', as_index=False).sum()
    ret_g = pd.merge(df, g, on='id', how='left')

    h = log_feature[['id', 'log_feature']].groupby(by='id', as_index=False).count()
    i = pd.merge(h, g, on='id', how='left')

    i.loc[:, 'feature_by_volume'] = i['log_feature'] * i['volume']
    ret = pd.merge(ret_g, i[['id', 'feature_by_volume']], on='id', how='left')

    return ret

In [20]:
def log_feature_prob(train, test, level=0):

    t = pd.merge(train[['id', 'fault_severity']], log_feature, on='id',
                 how='left').drop_duplicates()

    log_given_severity = \
       t.loc[t['fault_severity'] == level, 'log_feature'].value_counts() / \
       t['fault_severity'].value_counts()[level]

    severity_prob = \
        t['fault_severity'].value_counts()[level] / len(t)

    log_probs = \
        t['log_feature'].value_counts() / len(t)

    log_feature_probs = \
        (log_given_severity * severity_prob) / log_probs

    prob_df = pd.DataFrame({'probs': log_feature_probs})

    p = pd.merge(t, prob_df, left_on='log_feature', right_index=True, how='left')
    prob_table = pd.pivot_table(p, values='probs', index='id', columns='log_feature',
                                aggfunc=np.mean, fill_value=0)

    train = pd.merge(train, prob_table, left_on='id', right_index=True,
                     how='left')
    test = pd.merge(test, prob_table, left_on='id', right_index=True,
                    how='left')

    return train, test

In [21]:
def dangerous_log(train, test, level=0):

    feature_count = \
        train.loc[
            (train['fault_severity'] == level),
            ['log_feature', 'volume']
        ].groupby(by='log_feature', sort=False).sum()

    feature_total = \
        train.loc[
            (train['fault_severity'] == level),
            'volume'
        ].sum()

    danger = feature_count / feature_total

    train.loc[
        (train['fault_severity'] == level) & (train['log_feature'].isin(danger.index)),
            'dangerous_log'
    ] = danger['volume']

    test.loc[
        (test['fault_severity'] == level) & (test['log_feature'].isin(danger.index)),
            'dangerous_log'
    ] = danger['volume']

    return train, test

In [22]:
def danger_log(train, test):

    train, test = dangerous_log(train, test, level=0)
    train, test = dangerous_log(train, test, level=1)
    train, test = dangerous_log(train, test, level=2)

    return train, test

In [23]:
def base_features(df):

    df_event = event_type_features(df)
    df_resource = resource_type_features(df_event)
    df_severity = severity_type_features(df_resource)
    df_log = log_features(df_severity)

    df_log_count = log_feature_volume(df_log)
    df_severity_high = severity_high(df_log_count)
    df_resource_type = resource_type_event_count(df_severity_high)
    df_resource_type_log = resource_type_log(df_resource_type)

    df_complete = df_resource_type_log

    return df_complete

In [24]:
def location_features(train, test, cutoff=0):

    train_locations = train[['location']]
    test_locations = test[['location']]

    train_locations.loc[:, 'train'] = True
    test_locations.loc[:, 'train'] = False

    train_distribution = train_locations['location'].value_counts()

    locations = pd.concat([train_locations, test_locations])
    locations.loc[:, 'location'] = locations['location'] \
        .apply(rare_category, args=(train_distribution, ),
               cutoff=cutoff, value='RareLocation')

    locations_bin = pd.get_dummies(locations['location'])
    locations_dummy = pd.concat([locations, locations_bin], axis=1)

    msk = locations_dummy['train']
    locations_dummy.drop(['train', 'location'], axis=1, inplace=True)

    train_locs = pd.concat([train, locations_dummy[msk]], axis=1)
    test_locs = pd.concat([test, locations_dummy[~msk]], axis=1)

    return train_locs, test_locs

In [25]:
def dangerous_location(train, test):

    danger = \
        (train.loc[(train['fault_severity'] == 2) | (train['fault_severity'] == 1), 'location']).value_counts() / \
        len(train.loc[(train['fault_severity'] == 2) | (train['fault_severity'] == 1), 'location'])

    msk = danger >= 0.025

    train.loc[train['location'].isin(danger[msk].index), 'dangerous'] = 1
    test.loc[test['location'].isin(danger[msk].index), 'dangerous'] = 1

    return train, test

In [26]:
def event_type_features(df):

    event_type_dummies = pd.get_dummies(event_type['event_type'])
    event_dummy = pd.concat([event_type, event_type_dummies], axis=1)

    event_grpd = event_dummy \
        .groupby(event_dummy.id).sum()

    df_event = pd.merge(df, event_grpd, left_on='id', right_index=True)
    return df_event

In [27]:
def event_resource_features(df):

    e = pd.merge(event_type, resource_type, on='id')
    p = e.loc[e['event_type'] == 'event_type 23', ['id', 'resource_type']] \
        .groupby(by='id', as_index=False).count()
    df_23 = pd.merge(df, p, on='id', how='left')

    return df_23

In [28]:
def event_log_features(df):

    l = pd.merge(event_type, log_feature, on='id')
    p = l.loc[l['event_type'] == 'event_type 15', ['id', 'volume']] \
        .groupby(by='id', as_index=False).sum()

    df_15 = pd.merge(df, p[['id', 'volume']], on='id', how='left')

    p = l.loc[l['event_type'] == 'event_type 15', ['id', 'log_feature']] \
        .groupby(by='id', as_index=False).count()

    df_15c = pd.merge(df_15, p, on='id', how='left')


    p = l.loc[l['event_type'] == 'event_type 14', ['id', 'volume']] \
        .groupby(by='id', as_index=False).sum()
    df_14 = pd.merge(df_15c, p[['id', 'volume']], on='id', how='left')

    return df_14

In [29]:
def event_severity_prob(train, test, level=0):

    t = pd.merge(train[['id', 'fault_severity']], event_type, on='id',
                 how='left').drop_duplicates()

    event_given_severity = \
        t.loc[t['fault_severity'] == level, 'event_type'].value_counts() / \
        len(t.loc[t['fault_severity'] == level])

    severity_probs = \
        len(t.loc[t['fault_severity'] == level, :]) / len(t)

    event_probs = \
        t['event_type'].value_counts() / len(t['event_type'])

    event_severity_probs = \
        (event_given_severity * severity_probs) / event_probs

    prob_df = pd.DataFrame({'probs': event_severity_probs})

    p = pd.merge(t, prob_df, left_on='event_type', right_index=True, how='left')
    prob_table = pd.pivot_table(p, values='probs', index='id', columns='event_type',
                                aggfunc=np.median, fill_value=0)

    train = pd.merge(train, prob_table, left_on='id', right_index=True,
                     how='left')
    test = pd.merge(test, prob_table, left_on='id', right_index=True,
                    how='left')

    return train, test

In [30]:
def event_severity(train, test):

    train, test = event_severity_prob(train, test, level=2)

    return train, test

In [31]:
def rare_category(x, category_distribution, cutoff=1, value='Rare'):
    try:
        if category_distribution[x] < cutoff:
            return value
    except (ValueError, KeyError):
        return np.nan
    else:
        return x

In [32]:
def xgboost_model(train, labels, test):

    params = {}
    params['objective'] = 'multi:softprob'
    params['num_class'] = 3
    params['eval_metric'] = 'mlogloss'
    params['eta'] = 0.01
    params['gamma'] = 2.0
    params['max_depth'] = 8
    params['min_child_weight'] = 0.1
    params['max_delta_step'] = 1.5
    params['subsample'] = 0.75
    params['colsample_bytree'] = 0.85
    params['lambda'] = 0.2
    params['alpha'] = 0
    params['silent'] = 1

    xgtrain = xgb.DMatrix(train, labels)
    xgtest = xgb.DMatrix(test)

    num_rounds = 2400
    m = xgb.train(list(params.items()), xgtrain, num_rounds)
    return m, m.predict(xgtest)

# Modeling

In [ ]:
train = train_orig
test = test_orig

train_scores = []
scores = []

for train, test in train_test(train):
    train = base_features(train)
    test = base_features(test)
    train, test = location_features(train, test, cutoff=5)

    labels = train.fault_severity.values
    answers = test.fault_severity.values
    
    train.drop(['id', 'fault_severity', 'location'], axis=1, inplace=True)
    test.drop(['id', 'fault_severity', 'location'], axis=1, inplace=True)

    train = train.fillna(0)
    train = train.astype(float)
    test = test.fillna(0)
    test = test.astype(float)

    chi2 = GenericUnivariateSelect(score_func=chi2,
                                   mode='percentile',
                                   param=80)
    train = chi2.fit_transform(train, labels)
    test = chi2.transform(test)

    pca = PCA(n_components=400)
    train = pca.fit_transform(train)
    test = pca.transform(test)
    
    knn = KNeighborsClassifier(
        n_neighbors=15,
        weights='uniform',
        leaf_size=50,
        p=1,
        n_jobs=-1
    )
    knn.fit(train, labels)
    knn_train = np.reshape(knn.predict(train), (-1, 1))
    knn_test = np.reshape(knn.predict(test), (-1, 1))
    
    bayes = MultinomialNB(alpha=0,
                          fit_prior=True)
    bayes.fit(train, labels)
    bayes_train = np.reshape(bayes.predict(train), (-1, 1))
    bayes_test = np.reshape(bayes.predict(test), (-1, 1))

    svm = LinearSVC(
        penalty='l2',
        loss='hinge',
        C=1.0,
        random_state=42,
        intercept_scaling=5
    )
    svm.fit(train, labels)
    svm_train = np.reshape(svm.predict(train), (-1, 1))
    svm_test = np.reshape(svm.predict(test), (-1, 1))

    #train = np.append(train, knn_train, axis=1)
    #test = np.append(test, knn_test, axis=1)
    #train_labels = np.reshape(labels, (-1, 1))
    
    #train = np.append(train, train_labels, axis=1)
    #test = np.append(test, bayes_test, axis=1)
    
    #train = np.append(train, train_labels, axis=1)
    #test = np.append(test, svm_test, axis=1)
    
    model, predictions = xgboost_model(train, labels, test)
    train_score = sc.multi_log_loss(
        model.predict(xgb.DMatrix(train)), labels
    )
    train_scores.append(train_score)
    score = sc.multi_log_loss(predictions, answers)
    scores.append(score)

    print('cv train score', train_score)
    print('cv score:', score)
    
print('train score:', np.mean(train_scores))
print('score:', np.mean(scores))

In [ ]:
train = train_orig
test = test_orig

train = base_features(train_orig)
test = base_features(test_orig)
train, test = location_features(train, test, cutoff=5)
train, test = dangerous_location(train, test)
train, test = event_severity(train, test)

labels = train.fault_severity.values
ids = test['id']

train.drop(['id', 'fault_severity', 'location'], axis=1, inplace=True)
test.drop(['id', 'location'], axis=1, inplace=True)

train = train.fillna(0)
train = train.astype(float)

test = test.fillna(0)
test = test.astype(float)

ch2 = GenericUnivariateSelect(score_func=chi2,
                              mode='percentile',
                              param=80)
train = ch2.fit_transform(train, labels)
test = ch2.transform(test)

model, predictions = xgboost_model(train, labels, test)

train_score = multi_log_loss(
    model.predict(xgb.DMatrix(train)), labels
)
print('train score', train_score)

predictions_df = pd.DataFrame({
        'id': ids,
        'predict_0': predictions[:, 0],
        'predict_1': predictions[:, 1],
        'predict_2': predictions[:, 2]
    })
predictions_df.to_csv('{}_v{}.csv'.format('xgb', version),
                      index=False)

# Scratch

In [ ]:
# library(data.table)
# library(magrittr)
# library(stringr)
# library(doMC)
# library(caret)

# #
# remove0cols <- T
# writeFiles <- T
# computeFeatures <- T
# computeManualInteractions <- T
# groupCardinalities <- T

# log_feature_rarityThreshold <- 50
# event_type_rarity_threshold <- 5
# #

# source('/Users/cavagnolo/testing/delattore/utils.R')

# train <-
#   fread("/Users/cavagnolo/testing/delattore/data/train.csv") %>% setkey(id)
# test <-
#   fread("/Users/cavagnolo/testing/delattore/data/test.csv") %>% setkey(id)
# log_feature <-
#   fread("/Users/cavagnolo/testing/delattore/data/log_feature.csv") %>% setkey(id)
# event_type <-
#   fread("/Users/cavagnolo/testing/delattore/data/event_type.csv") %>% setkey(id)
# resource_type <-
#   fread("/Users/cavagnolo/testing/delattore/data/resource_type.csv") %>% setkey(id)
# severity_type <-
#   fread("/Users/cavagnolo/testing/delattore/data/severity_type.csv") %>% setkey(id)


# log_feature[, ":="(numlf = makeNumeric(log_feature), log_feature = makeReadable(log_feature))]
# event_type[, ":="(numet = makeNumeric(event_type), event_type = makeReadable(event_type))]
# resource_type[, ":="(resource_type = makeReadable(resource_type))]
# severity_type[, ":="(numst = makeNumeric(severity_type), severity_type = makeReadable(severity_type))]


# train[, ":="(numloc = makeNumeric(location),
#              location = makeReadable(location))]
# test[, ":="(numloc = makeNumeric(location),
#             location = makeReadable(location), fault_severity = -1)]

# total <- rbind(train, test)%>%setkey("id")

# if(groupCardinalities){
#   #LF reduction
#   rare_lf <- log_feature[total][,.N, by=log_feature][N<=log_feature_rarityThreshold][,log_feature]
#   log_feature$log_feature[log_feature$log_feature%in%rare_lf] <- "rare_lf"
  
#   rare_et <- event_type[total][,.N, by=event_type][N<=event_type_rarity_threshold][,event_type]
#   event_type$event_type[event_type$event_type%in%rare_et] <- "rare_et"
#   rm(list=c("rare_lf", "rare_et"))
# }

# if(computeFeatures){

#   t1 <- log_feature[total][,.(
#     loc_nid = uniqueN(id),
#     loc_nlf = uniqueN(log_feature),
#     loc_sumvol = sum(volume),
#     loc_avgvol = mean(volume),
#     loc_sdvol = ifelse(is.na(sd(volume)), 0, sd(volume)),
#     loc_logvol = log(sum(volume)),
#     loc_vollog = sum(log(volume+1)),
#     loc_sqrtvol = sqrt(sum(volume)),
#     loc_volsqrt = sum(sqrt(volume))), keyby=location]
  
#   t2 <- resource_type[total][, .(loc_nrt = uniqueN(resource_type)), keyby=location]
#   t3 <- event_type[total][, .(loc_net = uniqueN(event_type)), keyby=location]
#   t4 <- severity_type[total][, .(loc_nst = uniqueN(severity_type)), keyby=location]
  
#   joined_total <- resource_type[event_type][severity_type][total]
#   # nombres de combinaisons différentes de resource_type/event_type par location
#   t5 <- joined_total[,etrtcomb := paste(resource_type, event_type, sep = "x")][, .(loc_etrtcomb = uniqueN(etrtcomb)), keyby=location]
#   t6 <- joined_total[,etstcomb := paste(severity_type, event_type, sep = "x")][, .(loc_etstcomb = uniqueN(etstcomb)), keyby=location]
#   t7 <- joined_total[,rtstcomb := paste(severity_type, resource_type, sep = "x")][, .(loc_rtstcomb = uniqueN(rtstcomb)), keyby=location]
  
#   location_info_total <- t1[t2][t3][t4][t5][t6][t7]
  
#   # tests sur les log features en temps que numérique moyenne/id, somme/id, etc
#   lf_info <- log_feature[total][,.(
#     sumlf = sum(numlf), 
#     avglf = mean(numlf), 
#     sdlf = ifelse(is.na(sd(numlf)), 0, sd(numlf)),
#     minlf = min(numlf),
#     maxlf = max(numlf)),by=id]
  
#   # moyenne numerique de l'event_type * severity_type
#   etst <- event_type[, .(
#     id, 
#     minumet = min(numet), 
#     maxnumet = max(numet), 
#     avgnumet = mean(numet)), by=id][severity_type][total][,.(et.st = avgnumet*numst),by=id]
  
#   total <- total[lf_info][etst]
#   total <- merge(total, location_info_total, by = "location")
# }
# setkeyv(total[,":="(location=NULL)], c("id", "fault_severity"))

# # Y a-t-il des NA dans le jeu de train, si oui on droppe les lignes
# na.ids <- log_feature[total][fault_severity != -1][is.na(log_feature)][,id]
# if(length(na.ids) > 0)
#   total <- total[-na.ids]
# ####################################
# # dcast des data pour avoir une ligne par id

# # total_lf_volume  <- dcast(
# #   log_feature[total],
# #   id + fault_severity ~ log_feature,
# #   value.var = list("volume", "log_feature"),
# #   fun = list(sum, length)
# # )

# # on garde uniquement le volume
# total_lf_volume  <- dcast(
#   log_feature[total],
#   id + fault_severity ~ log_feature,
#   value.var = "volume",
#   fun = sum
# )

# total_et <-
#   dcast(event_type[total], id + fault_severity ~ event_type, value.var = "event_type", fun = length)
# total_rt <-
#   dcast(
#     resource_type[total], id + fault_severity ~ resource_type, value.var = "resource_type", fun = length
#   )
# total_st <-
#   dcast(
#     severity_type[total], id + fault_severity ~ severity_type, value.var = "severity_type", fun = length
#   )

# total.wide <-
#   total[total_lf_volume][total_et][total_rt][total_st]

# ######################################################################################
# #calculs d'intercations entre event_type, resource_types, log_features
# if(computeManualInteractions){
#   total.wide[,":="(

#     som_vol_test = (f203 * f312 * f232 * f170),
    
#     som_vol_feat = (f82 + f203 + f71 + f193 + f80),
#     som_vol_feat_c0 = (f313 + f233 + f315),
#     som_vol_feat_c1 = (f82 + f203 + f170),
#     som_vol_feat_c2 = (f71 + f193 + f80) ) ]
# }

# ######################################################################################
# train.wide <- total.wide[fault_severity != -1,-"id", with = FALSE]
# test.wide <- total.wide[fault_severity == -1,-"id", with = FALSE]

# # write files with train and test
# if(writeFiles){
#   writeLines("Writing train.csv and test.csv...")
#   write.csv(train.wide, paste(sep = "-", "train.csv"), row.names = F, quote = F)
#   write.csv(test.wide[,.SD, .SDcols = -"fault_severity"], paste(sep = "-", "test.csv"), row.names = F, quote = F)
#   writeLines("...done")}

# xtrain <- as.matrix(train.wide[,-"fault_severity", with = F])
# names(xtrain) <- setdiff(names(train.wide), "fault_severity")
# ytrain <- train.wide$fault_severity

# xtest <-  as.matrix(test.wide[,-"fault_severity", with = F])
# names(xtest) <- setdiff(names(test.wide), "fault_severity")
# test.id <- test$id

# # séparer le train set en 2 pour le blending
# # folds <- createFolds(train.wide$fault_severity, k = 3)
# # 80% train + 20% pour valider le stacking
# #if(onFold) folds <- createDataPartition(train.wide$fault_severity, p = 0.8)


# writeLines("Cleaning up...")
# rm(list=c("na.ids", "total.wide", "total_et", "total_rt", "total_st", 
#           "total_lf_volume", "train.wide", "test.wide", "total",
#           "location_info_total","joined_total", "lf_info", paste0("t", 1:7)))

In [ ]:
# set.seed(123456)

# if(onFold){
#   writeLines(paste("Starting xgboost on a fold1....",nrow(xtrain[folds$Resample1,]), " lines" ))
#   dtrain <- xgb.DMatrix(data = xtrain[folds$Resample1,], label = ytrain[folds$Resample1])
#   xtest <- xtrain[-folds$Resample1,]
#   test.id <- test.id[-folds$Resample1]
# }else{
#   writeLines("Starting xgboost on full train...")
#   dtrain <- xgb.DMatrix(data = xtrain, label = ytrain)
# }

# registerDoMC(cores = 6)

# xgparams.tree <- list(
#   objective = "multi:softprob",
#   num_class = 3,
#   colsample_bytree = 0.3,
#   max.depth = 8,
#   eta = 0.05
# )

# xgboost.first <- xgb.cv(
#   data = dtrain,
#   params = xgparams.tree,
#   nrounds = 500,
#   nfold = 10,
#   metrics = "mlogloss",
#   verbose = verboseXgboost,
#   print.every.n = 200
# )

# cat("xVal mlogloss : ", min(xgboost.first$test.mlogloss.mean),"\n")

# if(CVonly)
# {
#   pred.loop <- matrix(nrow = nrow(xtest)*3, ncol = 10)
#   for(index in 1:10)
#   {
#     set.seed(28021980+index)
#     xgboost.model <- xgboost(
#       data = dtrain,
#       params = xgparams.tree,
#       nrounds = which.min(xgboost.first$test.mlogloss.mean),
#       verbose = verboseXgboost
#     )
#     pred.loop[,index] <- xgboost::predict(xgboost.model, xtest)
#   }

#   pred.xgboost <- matrix(apply(pred.loop, MARGIN = 1, mean), ncol = 3, byrow = T)
  

# if(importance){
#   writeLines("Computing importance...")
#   imp <- xgb.importance(feature_names = names(xtrain), model = xgboost.model)
#   }
  
#   if (genererSubmission) {
#     cat("Generating Submission...\n")
#     output.xgboost <- data.frame(
#       id = test.id,
#       predict_0 = pred.xgboost[,1],
#       predict_1 = pred.xgboost[,2],
#       predict_2 = pred.xgboost[,3]
#     )
#     write.csv(output.xgboost, paste(sep = "-", format(Sys.time(), "%Y%m%d.%H%M"), "submission.csv"), row.names = F, quote = F)
#   }
# }